In [1]:
import os
os.chdir("../")
os.getcwd()

'/home/izam/coding/Loan-Prediction-System'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict
    target_col: str
    nan_ratio: float
    categories: dict

In [3]:
from loanPrediction.constants import *
from loanPrediction.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        target_col = self.schema.TARGET_COLUMN
        nan_ratio = self.schema.COL_NAN_RATIO
        categories = self.schema.CATEGORIES

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
            target_col=target_col.name,
            nan_ratio=nan_ratio,
            categories=categories,
        )

        return data_validation_config

In [5]:
import os
from loanPrediction import logger
import pandas as pd
from loanPrediction.utils.common import is_nan

In [6]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        self.data = pd.read_csv(self.config.unzip_data_dir)


    def _validate_all_columns(self)-> bool:
        try:
            all_cols = list(self.data.columns)
            all_schema = self.config.all_schema.keys()

            validation_status = True
            for col in all_schema:
                if col not in all_cols:
                    validation_status = False
                

            return validation_status
        
        except Exception as e:
            raise e
        
    def _validate_na_values(self) -> bool:
        try:
            total_rows = len(self.data)
            threshold = total_rows * self.config.nan_ratio
            all_cols = list(self.data.columns)
            na_values = self.data.isna().sum()

            validation_status = True
            for col in all_cols:
                if na_values[col] > threshold:
                    validation_status = False

            return validation_status

        except Exception as e:
            raise e

    def _validate_categories(self) -> bool:
        try:
            categories = self.config.categories
            data = self.data

            validation_status = True
            for col in categories:
                for category in data[col].unique():
                    if category not in categories[col] and is_nan(category)==False:
                        validation_status = False
            
            return validation_status

        except Exception as e:
            raise e
        
    def final_validation(self) -> bool:
        try:
            validation_column = self._validate_all_columns()
            validation_na = self._validate_na_values()
            validation_categories = self._validate_categories()

            if validation_column and validation_na and validation_categories:
                validation_all = True
            else:
                validation_all = False

            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation column status: {validation_column}\
                        \nValidation NA values status: {validation_na}\
                        \nValidation categorical columns: {validation_categories}\
                        \n\nValidation all: {validation_all}")
                

        except Exception as e:
            raise e

In [7]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.final_validation()
except Exception as e:
    raise e

[2023-12-11 12:30:41,430: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-11 12:30:41,433: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-11 12:30:41,437: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-11 12:30:41,438: INFO: common: created directory at: artifacts]
[2023-12-11 12:30:41,439: INFO: common: created directory at: artifacts/data_validation]
